In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [177]:
import torch, torch.nn as nn, torch.functional as F, torch.utils.data as data
import torchvision, numpy as np, os
import matplotlib.pyplot as plt
from fastai.conv_learner import ConvnetBuilder
from fastai.model import resnet34
import nvvl

In [185]:
PATH="/home/sufiyan/Common_data/mtp2/dataset/100_vids/"
PATH_L_Vids=PATH+"LEFT/"
PATH_L=PATH+"LEFT_data/"
PATH_L_L=PATH+"LEFT_labels/"
PATH_R_Vids=PATH+"RIGHT/"
PATH_R=PATH+"RIGHT_data/"
PATH_R_L=PATH+"RIGHT_labels/"

In [119]:
leftVids=[PATH_L+fileName for fileName in os.listdir(PATH_L) if os.path.isfile(PATH_L+fileName)]
leftVids_L=[PATH_L_L+fileName for fileName in os.listdir(PATH_L_L) if os.path.isfile(PATH_L_L+fileName)]

In [146]:
def vidLoader(vidsList):
    for vidName in vidsList:
        frames=np.load(vidName)
        for frame in frames:
            yield frame

In [145]:
print(len(np.load(leftVids[10])))        #number of frames
print(len(np.load(leftVids_L[10])))      #number of labels
img=(np.load(leftVids[20])[0])
print(img.shape)                         #shape of each frame
print(len(img.shape))                    #dimentions
# plt.imshow(img)

495
495
(224, 224, 1)
3


In [148]:
img=(np.load(leftVids[0])[0])            #Load a Frame
print(img.shape)                         #shape of each frame
print(len(img.shape))                    #Dimensions of frame

(224, 224, 3, 1)
4


Some video have 3 channels and are unsqueezed to 1 batch size whereas some video only have N\*M\*1 frame shape

In [163]:
#loads resnet34 pretrained on ImageNet using fastai library, Addaptive Pool 
#Followed by Fully connected layers of 500 and c
model=ConvnetBuilder(resnet34, c=3, is_multi=True, is_reg=False, pretrained=True).model

In [173]:
#string_to_int and int_to_string mapping
stoi={"w":0, "a":1, "d":2}; itos={0:"Clear", 1:"Left", 2:"Right"}

In [172]:
class myVidDataset(torch.utils.data.Dataset):
    def __init__(npArray, labelList):
        assert isinstance(npArray, np.ndarray), "npArray in myVidDataset should be a numpy array of Frames"
        assert len(npArray)==len(labelList), f"Length of Labels {len(labelList)} not equal to len \
                                                    of npArray {len(npArray)}"
        self.frames, self.labels = npArray,labelList
        
    def __getitem__(self, idx):
        x=self.frames[idx]
        x=np.squeeze(x)     #extra 1 dimention removed if any
        if x.shape[-1]==1 : x=np.dstack((x,x,x)) #id image is 1 channel, make it 3 channel
        return (x, self.labelList[idx])
    def __len__(self): return len(self.frames)

In [186]:
leftVids =[PATH_L_Vids+fileName for fileName in os.listdir(PATH_L_Vids) if os.path.isfile(PATH_L_Vids+fileName)]
rightVids=[PATH_R_Vids+fileName for fileName in os.listdir(PATH_R_Vids) if os.path.isfile(PATH_R_Vids+fileName)]
allVids=leftVids+rightVids
np.random.shuffle(allVids)

In [195]:
vidName=allVids[100].split("/")[-1]                 #Video Name
print(vidName)
print(".".join(vidName.split(".")[:-1]+["npy"]))    #Label File Name

FLIP_20170411_071853_Video.avi
FLIP_20170411_071853_Video.npy


In [196]:
def get_label(fileName, frameNum):
    vidName=fileName.split("/")[-1]
    labelName=".".join(vidName.split(".")[:-1]+["npy"])
    if fileName.split("/")[-2]=="RIGHT":
        label=list(np.load(PATH_R_L+labelName))[frameNum] 
    elif fileName.split("/")[-1]=="LEFT":
        label=list(np.load(PATH_L_L+labelName))[frameNum]
    else: 
        print(f"FileName: {fileName}   parent Folder not in [LEFT, RIGHT] returning 0 label for clear")
        label='w'
    return stoi[label] #return integer corrospondinf to label

In [198]:
allVids[0]

'/home/sufiyan/Common_data/mtp2/dataset/100_vids/RIGHT/FLIP_20170421_002950_Video.avi'

In [197]:
dataset=nvvl.VideoDataset(allVids, sequence_length=5, get_label=get_label)

FatalError: Unhandled codec 13 in /home/sufiyan/Common_data/mtp2/dataset/100_vids/RIGHT/FLIP_20170421_002950_Video.avi

The Above error isn't surprising as the video have been found to be super buggy. The Metadata suggests it has 500 frames but the video have generally been found to have 49x frames and in one case even 318 frames only. A good example below--

In [199]:
leftVids=[PATH_L+fileName for fileName in os.listdir(PATH_L) if os.path.isfile(PATH_L+fileName)]
len(np.load(leftVids[0]))

498

The Above video contains 498 frames rather than 500. This has been causing the problem from some time now.
apparently, I won't be able to use Nvidia's nvvl video dataset and VidelLoader... Will have to continue with numpy array of frames.

One way to go would be to decode the videos and reencode them with a new and better supported encoder like H.264. These newly encoded videos should work fine with nvvl.

In [211]:
#video decoded and encoded again with libx264 encoder
newVid= "/home/sufiyan/Common_data/mtp2/dataset/100_vids/newly_encoded/LEFT/output.mp4"

In [213]:
dataset=nvvl.VideoDataset([newVid], sequence_length=5, get_label=get_label)

Now the nvvl VideoDataset seems to work with the newly encoded videos. All the videos in the dataset will need to be re-encoded with H.264 to be able to use it with nvvl